## 1. Problem Background

In  contemporary  society,  the  environment  prob-lems especially heavy air pollution become one ofthe most severe menace for peoples health in China.Within a typical year in China, the haze (also calledsmog) weather accounts for almost 80 percent daysin a year. In particular, the main pollution materialis  called  PM  2.5,  which  as  its  name  suggests  hasa  diameter  less  than2.5μmand  can  be  easilyinhaled into humans body and cause serious disease.Therefore, as a group of Chinese graduate studentsin NYU, our team tries to predict the future PM 2.5index and to design a simple web interface to helppeople to plan their activities healthily and wisely.

## 2. Data Cleaning

In [ ]:
#Loading library 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
from statistics import mode,mean
import datetime
from sklearn import preprocessing
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler

In [ ]:


#Dataset loading
#https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data#
gz = pd.read_csv("GuangzhouPM20100101_20151231.csv")
gz.describe()

In [ ]:
print("NA count\n",gz.isnull().sum())
print("gz shape",gz.shape)
print("NA percent",max(gz.isnull().sum())/gz.shape[0])

In [ ]:
# #dataset Standardization and store information in dictionary for inversetransformation
# statis = {}
# def stands(x):
#     statis[x.name] = {}
#     statis[x.name]["mean"] = x.mean()
#     statis[x.name]["std"] = x.std()
#     x = (x-x.mean())/x.std()    
#     return x
# gz['DEWP'] = stands(gz['DEWP'])
# gz['HUMI'] = stands(gz['HUMI'])
# gz['PRES'] = stands(gz['PRES'])
# gz['TEMP'] = stands(gz['TEMP'])
# gz['Iws'] = stands(gz['Iws'])
# gz['precipitation'] = stands(gz['precipitation'])
# gz['Iprec'] = stands(gz['Iprec'])

In [ ]:
plt.figure(figsize=(15,12))
plt.subplot(311)
plt.scatter(gz['month'],gz['PM_City Station'], c='r', label = "City Station", marker=r'$\clubsuit$')
plt.legend(fontsize = 15)
plt.subplot(312)
plt.scatter(gz['month'],gz['PM_5th Middle School'],c='g', label = "5th Middle School", marker=(5, 2))
plt.legend(fontsize = 15)
plt.ylabel('PM2.5', fontsize=20)
plt.subplot(313)
plt.scatter(gz['month'],gz['PM_US Post'],c='b',label = "US Post", marker=(5, 0))
plt.legend(fontsize = 15)

plt.xlabel('month', fontsize=20)
plt.show()

In [ ]:
gz = gz.drop(['PM_City Station','PM_US Post'],axis = 1)


In [ ]:
print("NA count\n",gz.isnull().sum())
print("gz shape",gz.shape)
print("NA percent",max(gz.isnull().sum())/gz.shape[0])

In [ ]:
plt.plot(gz['PM_5th Middle School'], 'g')
plt.ylabel('PM2.5 reading', fontsize=15)
plt.xlabel('record index', fontsize=15)
plt.title('PM 2.5 at 5th Middle School',fontsize=20)
plt.show()

In [ ]:
gz['PM_5th Middle School'].describe()

In [ ]:
# gz['PM_5th Middle School'] = stands(pd.rolling_median(gz['PM_5th Middle School'],30))

In [ ]:
#Because frist 350000 row of index has too many NAN, we drop first 35000 row. 
plt.plot(gz['PM_5th Middle School'], 'g')
plt.ylabel('PM2.5 reading', fontsize=10)
plt.xlabel('index', fontsize=10)
plt.title('PM_5th Middle School',fontsize=20)
plt.show()
gz = gz.drop(gz.index[0:35000])

In [ ]:
plt.plot(gz['PM_5th Middle School'], 'g')
plt.ylabel('PM2.5 reading', fontsize=15)
plt.xlabel('Record index', fontsize=15)
plt.title('PM2.5 at 5th Middle School after removing',fontsize=20)
plt.show()

In [ ]:
gz['PM_5th Middle School'].describe()

In [ ]:
gz = gz.dropna(axis=0, how='any')
print("gz NA count\n",gz.isnull().sum())
print("gz shape",gz.shape)
print("NA percent",sum(gz.isnull().sum())/gz.shape[0])

In [ ]:
gz.to_csv('gzpm_cleaned.csv')

## 3.1 Data Modeling - Time Series

In [ ]:
data = pd.read_csv("gzpm_cleaned.csv", index_col=0)
data.head()

### 3.1.1 Stability test

In [ ]:
pmdt = data["PM_5th Middle School"]
plt.plot(pmdt, 'g')
plt.xlabel("label")
plt.ylabel("PM 2.5")
plt.show()

In [ ]:
import statsmodels.tsa.stattools as ts
result = ts.adfuller(np.array(pmdt))
result

From the ADF test result, we can see the p-value is $3e-22$, which is strong enough to reject the original hypertheiss. Plus, the value of this test is $-11.9$, which is smaller than the three threshold in the levels of$1\%, 10\%, 5\%$. So we can assume that our data is stationary.

### 3.1.2  Parameters Selection for ARIMA 

In [ ]:
import itertools
import warnings
import statsmodels.api as sma
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))
y = np.array(pmdt)
# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

warnings.filterwarnings("ignore") # specify to ignore warning messages

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sma.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

According to the AIC results, we select $ARIMA(1, 0, 1)*(0, 1, 1, 12)$ as our model.

### 3.1.3 Model Fitting and Predicting

In [ ]:
mod = sma.tsa.statespace.SARIMAX(y,
                                order=(1, 0, 1),
                                seasonal_order=(0, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

### 3.1.4 Visualization and Model Evaluation

In [ ]:
fig = plt.figure()
results.plot_diagnostics(figsize=(15, 12))
plt.show()

## 3.2 Data Modeling - Multi-Linear Regression

### 3.2.1 Correlation Analysis

In [ ]:
data.head()

In [ ]:
import seaborn as sns

f, ax = plt.subplots(figsize=(10, 8))
corr = data.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)
plt.show()

From the above figure, we can't see significant correlation between the target variable(PM 2.5) and hour, year, month, day. So we decide to discard those features.

### 3.2.2 Data Modelling

In [ ]:
data = data.join(pd.get_dummies(data['cbwd'], prefix='cbwd'))
data.drop('cbwd', axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop(["PM_5th Middle School", "year", "month", "day", "hour"], 1)
y = pd.DataFrame(data["PM_5th Middle School"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, y_train)


### 3.2.3 Prediction and Visualization

In [ ]:
# Plot outputs

y_pred = lm.predict(X_test)
y_predtr = lm.predict(X_train)
# plt.scatter(X_test, y_test,  color='black')
plt.figure()
ind = range(len(y_pred))
plt.subplot(121)
plt.plot(ind, list(y_pred), 'r', label = "Prediction")
plt.legend()
plt.xlabel("Index")
plt.ylabel("PM 2.5 Value")
plt.ylim((0,100))
plt.title("Linear Regression Prediction")
plt.subplot(122)
plt.plot(y_test.values, 'b',label = "Actual")
plt.legend()
plt.xlabel("Index")
plt.ylabel("PM 2.5 Value")
plt.ylim((0,300))
plt.title("Actual PM 2.5 Values")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_pred, y_test)

## 3.3 Data Modeling - Tree-based Regressor

### 3.3.1 Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
mse = []
maxdp = [6,8,10,12,14,16,18,20,22,24,26,28]
for dp in maxdp:
    dt = DecisionTreeRegressor(max_depth=dp)
    dt.fit(X_train, y_train)
    y_hat_dt = dt.predict(X_test)
    
    mse.append(mean_squared_error(y_hat_dt, y_test))



In [ ]:
plt.figure()
plt.plot(maxdp, mse, 'pink',linewidth = 3, marker=r'$\clubsuit$', label = "mse" )
plt.legend(fontsize = 15)
plt.xlabel("Max Depth")
plt.ylabel("MSE")
plt.show()

In [ ]:
mse

According to the figure above, we select 26 as the optimal max depth, and the MSE is 270.8.

### 3.3.2 Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

n_est = [50,100,150,200,250,300,350,400]
mse_rf = []
for n in n_est:
    regr = RandomForestRegressor(max_depth=26, random_state=0, n_estimators = n)
    regr.fit(X_train, y_train)
    y_hat_rf = regr.predict(X_test)
    print(n)
    mse_rf.append(mean_squared_error(y_hat_rf, y_test))
    
plt.figure()
plt.plot(n_est, mse_rf, 'orange',linewidth = 3, marker=r'$\clubsuit$', label = "mse" )
plt.legend(fontsize = 15)
plt.xlabel("Number of Estimators")
plt.ylabel("MSE")
plt.show()

In [ ]:
mse_rf

According to the figure above, we select 300 as the optimal number of estimators, and the MSE is 125.89. The performance of Random Forest Regressor is better than that of Decision Tree.

### 3.3.3 XGBoost Regressor

XGBoost here. Instead of plotting the results and selectting parameters manually, we use the powerful GridSearchCV to help us select optimal solution.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
xgb_model = xgb.XGBRegressor()
clf = GridSearchCV(xgb_model,
                   {'max_depth': [5,10, 15,20],
                    'n_estimators': [50,100,200,300]}, verbose=1)
clf.fit(X_train,y_train)
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
xgb_final = xgb.XGBRegressor(max_depth=10 ,n_estimators= 300 )
xgb_final.fit(X_train,y_train)
y_hat_xg = xgb_final.predict(X_test)
print("MSE:", mean_squared_error(y_hat_xg, y_test))

XGBoost helps us improve model performance, which reduce the testing MSE to 108.  

## 4. Data Preparation for Web Application Development

In [1]:
import numpy as np
import pandas as pd
import datetime
total_data = pd.read_csv('gzpm_cleaned.csv')
total_data.head(3)

,Unnamed: 0,No,year,month,day,hour,season,PM_5th Middle School,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec
0,35000,35001,2013,12,29,8,4.0,45.0,-7.0,40.0,1016.8,5.5,NE,5.8,0.0,0.0
1,35001,35002,2013,12,29,9,4.0,41.0,-7.0,34.0,1017.5,7.8,NW,1.9,0.0,0.0
2,35002,35003,2013,12,29,10,4.0,39.0,-7.1,30.0,1017.8,9.6,NE,3.5,0.0,0.0


In [ ]:
def find_data(year, month, day, hour):
    try:
        #print(year, month, day, hour)
        data_found = total_data.query('year == '+str(year)+' & day=='+str(day)+' \
                                      & hour=='+str(hour)+' & month=='+str(month))
        #print(data_found)
        if len(data_found)==1:
            return data_found
        else:
            return 'No Data'
    except:
        print('Worng Date')
        return 'No Data'

In [ ]:
find_data('2013','12','29','20')

In [ ]:
# The demo uses MLR to calculate the PM2.5 value, because it is easy to implement.
def calculate_PM25(df):
    #print(df)
    if len(df)>1:
        return 0
    coe1 = df['season'].values[0]
    coe2 = df['DEWP'].values[0]
    coe3 = df['HUMI'].values[0]
    coe4 = df['TEMP'].values[0]
    coe5 = df['cbwd'].values[0]
    coe6 = df['Iws'].values[0]
    coe7 = df['Iprec'].values[0]
    if coe1 == 1:
        coe1_value = -0.60931
    elif coe1 == 2:
        coe1_value = -1.083711
    elif coe1 == 3:
        coe1_value= -0.620802
    else:
        coe1_value = 0
    if coe5 == 'cv':
        coe5_value = 0.201879
    elif coe5 == 'NW':
        coe5_value = 0.250275
    else:
        coe5_value = 0
    PM25 = int(0.509180+coe1_value+(-10.348965)*coe2+2.006098*coe3+\
               0.778131*coe4+coe5_value+(-0.077538)*coe6+(-0.061016)*coe7) + 1
    if PM25 < 0:
        return 0
    else:
        return PM25
    

In [ ]:
def delete_zero(string):
    if string[0]=='0':
        return string[1:]
    else:
        return string
    
def find_next_day(year, month, day):
    next_day = datetime.date(int(year),int(month),int(day))+datetime.timedelta(days=+1)
    next_day = str(next_day).split('-')
    return next_day[0], delete_zero(next_day[1]), delete_zero(next_day[2]) 

#def calculate(year, month, day, hour):
#    current_data = find_data(year, month, day, hour)
#    next_year, next_month, next_day = find_next_day(year, month, day)
    #print(next_day)
#    next_data = find_data(next_year, next_month, next_day, hour)
#    next_year, next_month, next_day  = find_next_day(next_year, next_month, next_day )
#    next_next_data = find_data(next_year, next_month, next_day, hour)
    #print(current_data, next_data, next_next_day)
#    if len(current_data) != 1 or len(next_data) != 1 or len(next_next_data) != 1:
#        return
    
    #a = current_data['PRES'].values[0]
    #b = current_data['Iws'].values[0]
    #return 1.8*a+7*b
#calculate(2012,2,29,1)

In [ ]:
calculate_PM25(find_data('2013','12','29','20'))

## 5. Web Interface Development

In [ ]:
#Thepart generate the HTML page, the content will be different based on the url(date) entered
def html_header_generator():
    text = '''<!DOCTYPE html>
<html>
<head>
<title>PM2.5 in Guang Zhou</title>
<link rel="stylesheet" type="text/css" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css">
<link rel="stylesheet" type="text/css" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesome.min.css">
<link rel="stylesheet" type="text/css" href="css/style.css" />
<link href="http://fonts.googleapis.com/css?family=Dosis:200,300,400,500,600" rel="stylesheet">
<link href="http://fonts.googleapis.com/css?family=Open+Sans:300,300i,400,400i,600,600i,700,700i" rel="stylesheet">
</head>
<body>
			<div class="container">
				 <h3>PM2.5 In Guang Zhou</h3>
						<div class="container">'''
    return text

def html_data_generator(year, month, day, hour, index):
    if index == 0:
        try: pm_25 = int(find_data(year, month, day, hour)['PM_5th Middle School'].values[0])
        except:return ''
    else:
        try: pm_25 = calculate_PM25(find_data(year, month, day, hour))
        except:return ''
    current_day = datetime.date(int(year),int(month),int(day))
    if index == 0:
        Day = 'Today'
    if index == 1:
        Day = 'Tomorrow'
    if index == 2:
        Day = '2 Days Later'
    if index == 3:
        Day = '3 Days Later'
    if pm_25<100:
        identifier = '''<div class="col-md-3">
					
						
							<h4>'''+Day+'''</h4>
							<h3>'''+str(pm_25)+'''</h3>
							<h5>'''+str(current_day)+'''</h5>
						
						<div class="value-gd-bottom">
							<button type="button" class="btn btn-success" onclick="location.href = 'https://www.tripadvisor.com/Tourism-g298555-Guangzhou_Guangdong-Vacations.html'">Outdoor</button>
						</div>
				</div>'''
    else:
        identifier = '''<div class="col-md-3">
					
						<div class="value">
							<h4>'''+Day+'''</h4>
							<h3>'''+str(pm_25)+'''</h3>
							<h5>'''+str(current_day)+'''</h5>
						
						<div class="value-gd-bottom">
                            <button type="button" class="btn btn-danger" onclick="location.href = 'https://www.amazon.com/s/ref=nb_sb_noss_2?url=search-alias%3Daps&field-keywords=PM2.5&rh=i%3Aaps%2Ck%3APM2.5'">Indoor</button>
						</div>
                        
					</div>
				</div>'''
    return identifier
        
def html_foot_generator():
    text = '''
			</div>
		</div>
			<div>
				    <p>Copyright &copy; 2017. NYU CDS DS-GA-1007.</p>
			</div>
<script src="https://code.jquery.com/jquery-2.1.4.js"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/js/bootstrap.min.js"></script>
</body>
</html>
'''
    return text
  

In [ ]:
#This part generate the Server
import http.server
import socketserver

#PORT = 8000

#Handler = http.server.SimpleHTTPRequestHandler




#import BaseHTTPServer
from socketserver import BaseServer, ThreadingTCPServer, TCPServer
import threading
from socketserver import ThreadingMixIn
import urllib
import os

class WebRequestHandler(http.server.SimpleHTTPRequestHandler):

    def do_GET(self):
        if "ico" in self.path:
            return
        thread = threading.currentThread()
        self.send_response(200)
        print(self.path)
        
        if "?" in self.path:
            self.send_header("Content-type","text/html")
            self.end_headers()
            if '?' not in self.path or '-' not in self.path:
                self.wfile.write('Worong Format'.encode())
            elif len(self.path.split('?')[1].split('-'))!=3 and len(self.path.split('?')[1].split('-'))!=4  :
                self.wfile.write('Worong Format'.encode())
                #current_date
            else:
                today = self.path.split('?')[1].split('-')
                today_1 = today[0]
                today_2 = delete_zero(today[1])
                today_3 = delete_zero(today[2])
                if len(today) ==4:
                    hour = delete_zero(today[3])
                else:
                    hour = 0
                tomorrow_1, tomorrow_2, tomorrow_3 = find_next_day(today_1, today_2, today_3)
                two_days_later_1, two_days_later_2, two_days_later_3 = find_next_day(tomorrow_1, tomorrow_2, tomorrow_3)
                three_days_later_1, three_days_later_2, three_days_later_3 = find_next_day(two_days_later_1, two_days_later_2, two_days_later_3)
                text_1 = html_header_generator()
                #print(text_1)
                text_2 = html_data_generator(today_1, today_2, today_3, hour, 0)
                #print(text_2)
                text_3 = html_data_generator(tomorrow_1, tomorrow_2, tomorrow_3 , hour, 1)
                #print(text_3)
                text_4 = html_data_generator(two_days_later_1, two_days_later_2, two_days_later_3, hour, 2)
                #print(text_4)
                text_5 = html_data_generator(three_days_later_1, three_days_later_2, three_days_later_3, hour, 3)
                #print(text_5)
                text_6 = html_foot_generator()
                #print(text_6)
                final_text = text_1 + text_2 + text_3 + text_4 + text_5 + text_6
                self.wfile.write(final_text.encode())
                
                
        elif 'css' in self.path:
            self.send_header("Content-type","text/css")
            self.end_headers()
        #self.send_header(“href”,”css/style.css”)
            html_object = open(self.path[1:])
            try:
                all_the_text = html_object.read()
            finally:
                html_object.close()
        #output = "<html><head><title>PM2.5</title></head><body><p>Three days PM2.5: 56, 78, 91</p></body></html>"
        #print(all_the_text)    
            self.wfile.write(all_the_text.encode())




class ThreadingServer(ThreadingMixIn, TCPServer):
    pass

if __name__ == '__main__':
    #server = TCPServer(('127.0.0.1', 6689), WebRequestHandler)
    #try:
    #    server.serve_forever()
    #except:
    #    pass
    #server.server_close()
    server = ThreadingServer(('127.0.0.1', 6671), WebRequestHandler)
    thread = threading.Thread(target=server.serve_forever)
    thread.start()
    print ("Server Starts")
    while True:
        try:
            continue
        except:
            print('Server Stops')
            break

Access by entering the URL with following format:localhost:6671/?year-month-day or localhost:6671/?year-month-day-hour
* For example: [localhost:6671/?2013-01-16-4](127.0.0.1:6671/?2013-01-16-4)
* To clearly see the performance, another two css files are needed.